In [2]:
#%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch                    
import torchvision
import fasttext

In [3]:
from transformers.modeling_bert import BertModel, BertForMaskedLM

from mmf.utils.build import (
    build_classifier_layer,
    build_image_encoder,
    build_text_encoder,
)

from omegaconf import OmegaConf

import os
#from mmf.utils.configuration import ( load_yaml_with_defaults, get_zoo_config, get_config)
import mmf.utils.configuration


In [4]:
#create dataset for vilbert
#future note - this oads the dataset as a global variable so no further changes can be made


from mmf.common.registry import registry
from mmf.utils.build import build_dataset
from mmf.utils.configuration import Configuration
from mmf.utils.configuration import load_yaml
import argparse
from mmf.utils.flags import flags
parser = flags.get_parser()

from mmf.utils.env import setup_imports
setup_imports()
args_str = ""

opts = ["model=vilbert", "config=../mmf/configs/datasets/hateful_memes/with_features.yaml", "run_type=val",
        "checkpoint.resume_zoo=vilbert.finetuned.hateful_memes.from_cc_original",
        "dataset=hateful_memes", "dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_seen.jsonl", "dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_seen.jsonl"]
args = argparse.Namespace(config_override=None)
args.opts = opts
print(args)
configuration = Configuration(args)
dataset_config = configuration.get_config()

model_data_processing_config = load_yaml("../mmf/projects/hateful_memes/configs/vilbert/defaults.yaml")
dataset_config.dataset_config.hateful_memes.use_images = False
print(dataset_config.dataset_config.hateful_memes)

#replace origina dataset processors with vilbert processors
dataset_config.dataset_config.hateful_memes.processors = model_data_processing_config.dataset_config.hateful_memes.processors

print(dataset_config.dataset_config.hateful_memes)
dataset = build_dataset("hateful_memes", dataset_config.dataset_config.hateful_memes)

#sanity
print(dataset.__getitem__(0))

#for visualization - relevant for images loading
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (20, 20)
#dataset.visualize(num_samples=8, size=(512, 512), nrow=4)


Namespace(config_override=None, opts=['model=vilbert', 'config=../mmf/configs/datasets/hateful_memes/with_features.yaml', 'run_type=val', 'checkpoint.resume_zoo=vilbert.finetuned.hateful_memes.from_cc_original', 'dataset=hateful_memes', 'dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_seen.jsonl', 'dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_seen.jsonl'])


/Users/efrat/source/dl_course/mmf/mmf/utils/configuration.py:536: UserWarning: Device specified is 'cuda' but cuda is not present. Switching to CPU version.
  "Device specified is 'cuda' but cuda is not present. "
/Users/efrat/anaconda3/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (model.bert) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


{'data_dir': '/Users/efrat/.cache/torch/mmf/data/datasets', 'depth_first': False, 'fast_read': False, 'use_images': False, 'use_features': True, 'return_features_info': True, 'images': {'train': ['hateful_memes/defaults/images/'], 'val': ['hateful_memes/defaults/images/'], 'test': ['hateful_memes/defaults/images/']}, 'features': {'train': ['hateful_memes/defaults/features/detectron.lmdb'], 'val': ['hateful_memes/defaults/features/detectron.lmdb'], 'test': ['hateful_memes/defaults/features/detectron.lmdb']}, 'annotations': {'train': ['hateful_memes/defaults/annotations/train.jsonl'], 'val': ['hateful_memes/defaults/annotations/dev_seen.jsonl'], 'test': ['hateful_memes/defaults/annotations/test_seen.jsonl']}, 'max_features': 100, 'processors': {'text_processor': {'type': 'vocab', 'params': {'max_length': 14, 'vocab': {'type': 'intersected', 'embedding_name': 'glove.6B.300d', 'vocab_file': 'hateful_memes/defaults/extras/vocabs/vocabulary_100k.txt'}, 'preprocessor': {'type': 'simple_senten

In [31]:
import pandas as pd
import subprocess
import csv

print(Path.cwd())
data_dir = Path("/Users/efrat/.cache/torch/mmf/data/datasets/hateful_memes/defaults")
annotations_dir = data_dir / "annotations" 


img_path = data_dir / "images" 
train_path = annotations_dir / "train.jsonl"
dev_path = annotations_dir / "dev_seen.jsonl"
test_path = annotations_dir / "test_seen.jsonl"

train_samples_frame = pd.read_json(train_path, lines=True)
train_samples_frame.head()

from PIL import Image

images = [
    Image.open(
        img_path / train_samples_frame.loc[i, "img"]
    ).convert("RGB")
    for i in range(10)
]

for image in images:
    print(image.size)
    


/Users/efrat/source/dl_course/my_hateful_memes
(265, 400)
(800, 533)
(558, 800)
(693, 800)
(550, 416)
(550, 341)
(800, 533)
(825, 618)
(550, 366)
(177, 800)
/Users/efrat/source/dl_course/my_hateful_memes
curl -F "image=@../content/images/img/42953.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie holding a microphone .


In [33]:
#write image captioning files for the dataset
def write_image_captioning(samples_frame, filename):
    input_shape = samples_frame.shape    

    # field names  
    fields = ['id', 'image_text']  
    
    # writing to csv file  
    with open(filename, 'w') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerow(fields) 

        for i in range(input_shape[0]):
            image_name =  samples_frame.loc[i, "img"]
            input = 'curl -F "image=@../content/images/' +image_name + '" -X POST http://localhost:5000/model/predict'
            print(input)
            proc = subprocess.Popen([input], stdout=subprocess.PIPE, shell=True)
            (out, err) = proc.communicate()
            out_as_dict = json.loads(out)
            print(out_as_dict['predictions'][0]['caption'])
            # writing the fields  
            csvwriter.writerow([samples_frame.loc[i, "id"],out_as_dict['predictions'][0]['caption']])  
            
dev_samples_frame = pd.read_json(dev_path, lines=True)
dev_samples_frame.head()

test_samples_frame = pd.read_json(test_path, lines=True)
test_samples_frame.head()

filename = "image_captioning_train.csv"
write_image_captioning(train_samples_frame, filename)

filename = "image_captioning_dev.csv"
write_image_captioning(dev_samples_frame, filename)

filename = "image_captioning_test.csv"
write_image_captioning(test_samples_frame, filename)


curl -F "image=@../content/images/img/42953.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie holding a microphone .
curl -F "image=@../content/images/img/23058.png" -X POST http://localhost:5000/model/predict
a couple of men standing next to each other .
curl -F "image=@../content/images/img/13894.png" -X POST http://localhost:5000/model/predict
a cat is wearing a santa hat and a tie .
curl -F "image=@../content/images/img/37408.png" -X POST http://localhost:5000/model/predict
a black and brown dog sitting on a chair .
curl -F "image=@../content/images/img/82403.png" -X POST http://localhost:5000/model/predict
a black and white photo of a man in a suit
curl -F "image=@../content/images/img/16952.png" -X POST http://localhost:5000/model/predict
a group of men playing a game of frisbee .
curl -F "image=@../content/images/img/76932.png" -X POST http://localhost:5000/model/predict
a couple of brown bears standing next to each other .
curl -F "image=@../content/image

a group of people sitting around a table .
curl -F "image=@../content/images/img/69357.png" -X POST http://localhost:5000/model/predict
a man in a baseball uniform holding a baseball bat .
curl -F "image=@../content/images/img/54602.png" -X POST http://localhost:5000/model/predict
a man and a dog are standing on a skateboard .
curl -F "image=@../content/images/img/15320.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie holding a microphone .
curl -F "image=@../content/images/img/63182.png" -X POST http://localhost:5000/model/predict
a close up of a person holding a banana
curl -F "image=@../content/images/img/20619.png" -X POST http://localhost:5000/model/predict
a man sitting in a chair holding a skateboard .
curl -F "image=@../content/images/img/82457.png" -X POST http://localhost:5000/model/predict
a woman holding a cell phone in her hand .
curl -F "image=@../content/images/img/81059.png" -X POST http://localhost:5000/model/predict
a group of young children si

a close up of a person holding a pair of scissors
curl -F "image=@../content/images/img/12890.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie standing on a beach .
curl -F "image=@../content/images/img/54102.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie standing in front of a microphone .
curl -F "image=@../content/images/img/63217.png" -X POST http://localhost:5000/model/predict
a woman wearing a white shirt and tie .
curl -F "image=@../content/images/img/03271.png" -X POST http://localhost:5000/model/predict
a man holding a cell phone up to his ear .
curl -F "image=@../content/images/img/35264.png" -X POST http://localhost:5000/model/predict
a little girl wearing a blue shirt and tie .
curl -F "image=@../content/images/img/63710.png" -X POST http://localhost:5000/model/predict
a group of people sitting around a table .
curl -F "image=@../content/images/img/12348.png" -X POST http://localhost:5000/model/predict
a man in a suit and ti

a car is parked in front of a parking meter .
curl -F "image=@../content/images/img/76250.png" -X POST http://localhost:5000/model/predict
a group of people standing next to each other .
curl -F "image=@../content/images/img/26198.png" -X POST http://localhost:5000/model/predict
a baseball player holding a bat on a field .
curl -F "image=@../content/images/img/61952.png" -X POST http://localhost:5000/model/predict
a woman is holding a wii remote in her hand .
curl -F "image=@../content/images/img/80269.png" -X POST http://localhost:5000/model/predict
a black and white photo of a baseball team .
curl -F "image=@../content/images/img/05712.png" -X POST http://localhost:5000/model/predict
a woman standing in front of a statue of a man .
curl -F "image=@../content/images/img/72941.png" -X POST http://localhost:5000/model/predict
a fire hydrant in the middle of a forest .
curl -F "image=@../content/images/img/21853.png" -X POST http://localhost:5000/model/predict
a car parked in front of a 

a man standing in the snow with a snowboard .
curl -F "image=@../content/images/img/43617.png" -X POST http://localhost:5000/model/predict
a group of people standing on top of a boat .
curl -F "image=@../content/images/img/70942.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie holding a microphone .
curl -F "image=@../content/images/img/49601.png" -X POST http://localhost:5000/model/predict
a man in a suit talking on a cell phone .
curl -F "image=@../content/images/img/35102.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie standing in a room .
curl -F "image=@../content/images/img/10659.png" -X POST http://localhost:5000/model/predict
a woman holding a cake with candles on it .
curl -F "image=@../content/images/img/36081.png" -X POST http://localhost:5000/model/predict
a birthday cake with a train on it
curl -F "image=@../content/images/img/68325.png" -X POST http://localhost:5000/model/predict
a woman is holding a cell phone to her ear .

...

a black and white photo of a man in a suit
curl -F "image=@../content/images/img/23897.png" -X POST http://localhost:5000/model/predict
a fire hydrant in the middle of a street .
curl -F "image=@../content/images/img/62031.png" -X POST http://localhost:5000/model/predict
a woman taking a picture of herself in a mirror .
curl -F "image=@../content/images/img/89106.png" -X POST http://localhost:5000/model/predict
a person flying a kite in the sky .
curl -F "image=@../content/images/img/84652.png" -X POST http://localhost:5000/model/predict
a group of people sitting at a table with wine glasses .
curl -F "image=@../content/images/img/14836.png" -X POST http://localhost:5000/model/predict
a man is on a skateboard in the air .
curl -F "image=@../content/images/img/05864.png" -X POST http://localhost:5000/model/predict
a group of giraffe standing next to each other .
curl -F "image=@../content/images/img/94107.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie standing 

a woman in a white dress shirt and tie .
curl -F "image=@../content/images/img/81309.png" -X POST http://localhost:5000/model/predict
a group of young men standing next to each other .
curl -F "image=@../content/images/img/41527.png" -X POST http://localhost:5000/model/predict
a man holding a banana up to his face .
curl -F "image=@../content/images/img/89250.png" -X POST http://localhost:5000/model/predict
a black and white photo of a woman sitting on a bench .
curl -F "image=@../content/images/img/98632.png" -X POST http://localhost:5000/model/predict
a group of men standing next to each other .
curl -F "image=@../content/images/img/29408.png" -X POST http://localhost:5000/model/predict
a truck driving down a road next to a road .
curl -F "image=@../content/images/img/38496.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie is smiling .
curl -F "image=@../content/images/img/06195.png" -X POST http://localhost:5000/model/predict
a man standing in a kitchen next t

a woman wearing a blue shirt and black tie .
curl -F "image=@../content/images/img/16529.png" -X POST http://localhost:5000/model/predict
a young girl holding a frisbee in her hands .
curl -F "image=@../content/images/img/89107.png" -X POST http://localhost:5000/model/predict
a man holding an umbrella in a field .
curl -F "image=@../content/images/img/42308.png" -X POST http://localhost:5000/model/predict
a little girl sitting at a table with a cake .
curl -F "image=@../content/images/img/19684.png" -X POST http://localhost:5000/model/predict
a couple of dogs that are laying down
curl -F "image=@../content/images/img/31627.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie holding a teddy bear .
curl -F "image=@../content/images/img/56184.png" -X POST http://localhost:5000/model/predict
a woman in a white shirt and black tie
curl -F "image=@../content/images/img/32965.png" -X POST http://localhost:5000/model/predict
a group of people standing in a room .
curl -F "

a man and woman sitting on a park bench .
curl -F "image=@../content/images/img/17539.png" -X POST http://localhost:5000/model/predict
a man holding a pair of scissors in his hand .
curl -F "image=@../content/images/img/92783.png" -X POST http://localhost:5000/model/predict
a group of people standing around a tennis court .
curl -F "image=@../content/images/img/86970.png" -X POST http://localhost:5000/model/predict
a woman is standing in front of a mirror .
curl -F "image=@../content/images/img/21903.png" -X POST http://localhost:5000/model/predict
a group of people standing next to each other .
curl -F "image=@../content/images/img/51890.png" -X POST http://localhost:5000/model/predict
a man is holding a baseball bat in his hands .
curl -F "image=@../content/images/img/34678.png" -X POST http://localhost:5000/model/predict
a man riding a bike with a dog on it .
curl -F "image=@../content/images/img/91740.png" -X POST http://localhost:5000/model/predict
a bunch of birds that are hangin

a man in a suit and tie standing in a room .
curl -F "image=@../content/images/img/31985.png" -X POST http://localhost:5000/model/predict
a man in a black shirt is talking on a cellphone
curl -F "image=@../content/images/img/07314.png" -X POST http://localhost:5000/model/predict
a woman and a dog sitting on a couch .
curl -F "image=@../content/images/img/91275.png" -X POST http://localhost:5000/model/predict
a cow is standing in a field of grass .
curl -F "image=@../content/images/img/95260.png" -X POST http://localhost:5000/model/predict
a young boy holding a baseball bat on a field .
curl -F "image=@../content/images/img/21364.png" -X POST http://localhost:5000/model/predict
a group of men standing next to each other .
curl -F "image=@../content/images/img/35210.png" -X POST http://localhost:5000/model/predict
a man in a suit and tie is smiling .
curl -F "image=@../content/images/img/07451.png" -X POST http://localhost:5000/model/predict
a black and white photo of a tennis team .
cur

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
index_of_data = 0
plt.imshow(images[index_of_data])
plt.axis("off")
plt.show()
print(train_samples_frame.loc[index_of_data, "text"])
#model.classify(images[7],train_samples_frame.loc[7, "text"])

In [8]:
import matplotlib.pyplot as plt
import requests
from mmf.utils.build import (
    build_model, build_processors
)

from PIL import Image

from mmf.models.vilbert import ViLBERTBase
from vilbert_with_tokens import ViLBERTWithTokens
from transformers import BertConfig
from omegaconf import OmegaConf
#from mmf.models.vilbert import ViLBERTForClassification


#model = MMBT.from_pretrained("mmbt.hateful_memes.images")

model_cls = registry.get_model_class("vilbert")
print(model_cls)
model = model_cls.from_pretrained("vilbert.finetuned.hateful_memes.from_cc_original")
#model = model_cls.from_pretrained("vilbert.pretrained.cc.original")
model.build()

#vilbert_config = load_yaml("../mmf/mmf/configs/models/vilbert/defaults.yaml")
#print("**********************")
#print(vilbert_config)
#vilbert_config.model_config.vilbert.training_head_type = "classification"
#print("*************************")
#model = ViLBERTBase.from_pretrained(vilbert_config.model_config.vilbert.bert_model_name, \
#                                        config=BertConfig.from_dict(OmegaConf.to_container(vilbert_config.model_config.vilbert, resolve=True)))
#mmf_model_csl = registry.get_model_class("mmf_transformer")
#mmf_model = mmf_model_csl.from_pretrained("mmf_transformer")
#mmf_model.build()

#processor_dict = build_processors(config.dataset_config.hateful_memes.processors)

#model = ViLBERTWithTokens.from_pretrained("vilbert.finetuned.hateful_memes.direct")
#config = get_config("../mmf/projects/vilbert/configs/hateful_memes/defaults.yaml")
#model = build_model(config.model_config)

<class 'vilbert_with_tokens.ViLBERTWithTokens'>


In [9]:
from transformers.tokenization_auto import AutoTokenizer

from mmf.utils.build import (
    build_dataloader_and_sampler
)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda(0)


#config = get_config("../mmf/projects/vilbert/configs/hateful_memes/defaults.yaml")
# Load pre-trained model tokenizer (vocabulary)
bert_model_version = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_version)
print(dataset_config.training)
dataset_config.training.num_workers = 0
dataset_config.training.batch_size = 1
dataloader, sampler = build_dataloader_and_sampler(dataset, dataset_config.training)

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent)+[0]*(128-len(sent)) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]
print(mask_id)
print(sep_id)
print(cls_id)

{'trainer': 'mmf', 'seed': -1, 'experiment_name': 'run', 'max_updates': 22000, 'max_epochs': None, 'log_interval': 100, 'logger_level': 'info', 'log_format': 'simple', 'log_detailed_config': False, 'should_not_log': False, 'colored_logs': True, 'tensorboard': False, 'batch_size': 512, 'update_frequency': 1, 'num_workers': 4, 'fast_read': False, 'dataset_size_proportional_sampling': True, 'pin_memory': False, 'checkpoint_interval': 1000, 'evaluation_interval': 1000, 'clip_gradients': False, 'clip_norm_mode': 'all', 'early_stop': {'enabled': False, 'patience': 4000, 'criteria': 'total_loss', 'minimize': True}, 'lr_scheduler': False, 'lr_steps': [], 'lr_ratio': 0.1, 'use_warmup': False, 'warmup_factor': 0.2, 'warmup_iterations': 1000, 'device': 'cpu', 'local_rank': None, 'verbose_dump': False, 'find_unused_parameters': False, 'evaluate_metrics': False, 'detect_anomaly': False, 'fp16': False}
103
102
101


In [10]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=0, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:        
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    print(idx.tolist())
    return idx.tolist() if return_list else idx

In [11]:
# Generation modes as functions
import math
import time
import tqdm

def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    #if rand_init:
    #    for ii in range(max_len):
    #        init_idx[seed_len+ii] = np.random.randint(0, len(tokenizer.vocab))
    
    return tokenize_batch(batch)

def parallel_sequential_generation(original_batch, seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    
    print("batch_size: ") 
    prepared_batch = dataset.prepare_batch(original_batch)
    print(prepared_batch.get_batch_size())
    batch_size = prepared_batch.get_batch_size()
    print(prepared_batch.fields())
    init_text_ids = get_init_text(seed_text, max_len, batch_size)
    print(init_text_ids)
    print("text_before:")
    print(prepared_batch.get_field("text"))
    print("input ids before:")
    print(prepared_batch.get_field("input_ids"))

   # check = dataset.__getitem__(0)          
    text_inp = torch.tensor(init_text_ids)
    prepared_batch.add_field("input_ids", text_inp)


    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            prepared_batch.get_field("input_ids")[jj][seed_len+kk] = mask_id

        print("input ids after:")
        print(prepared_batch.get_field("input_ids")) 
        print("-----------------------------------------------")
        out = model(prepared_batch)
        print(model.children())
        state = model.state_dict
        t_pooler_layer = state()['model.bert.t_pooler.dense.weight']

    #    print(out)
        print(t_pooler_layer)
     #   print("now getting image captioning only")
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(t_pooler_layer, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        #print(idxs.type())
        input_ids_batch = prepared_batch.get_field("input_ids").numpy()
        print(input_ids_batch)

        for jj in range(batch_size):
            input_ids_batch[jj][seed_len+kk] = idxs#idxs[jj]

        prepared_batch.add_field("input_ids", torch.from_numpy(input_ids_batch))

        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(prepared_batch.get_field("input_ids").numpy()[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
    return untokenize_batch(prepared_batch.get_field("input_ids"))    

def parallel_generation(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at a time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=seed_len+kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii, print_every) == 0:
            print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(original_batch, seed_text, batch_size=2, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    batch = batch.cuda() if cuda else batch
    
    prepared_batch = dataset.prepare_batch(original_batch)
    print(prepared_batch.get_batch_size())
    batch_size = prepared_batch.get_batch_size()
    init_text_ids = get_init_text(seed_text, max_len, batch_size)
    print(init_text_ids)
    print("text_before:")
    print(prepared_batch.get_field("text"))
    print("input ids before:")
    print(prepared_batch.get_field("input_ids"))
    text_inp = torch.tensor(init_text_ids)
    prepared_batch.add_field("input_ids", text_inp)
    print("input ids after:")
    print(prepared_batch.get_field("input_ids")) 
    print("-----------------------------------------------")


    for ii in range(max_len):
        before = prepared_batch.get_field("input_ids")
        before[0][seed_len+ii+leed_out_len] = sep_id
       #does it work automatically? 
      #  inp = [sent[:seed_len+ii+leed_out_len]+[sep_id] for sent in batch]
      #  iprepared_batch.get_field("input_ids")np = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        print(prepared_batch.get_field("input_ids"))
        #new_model = nn.Sequential(*list(model.children())[:-1])
        out = model(prepared_batch)
        print(out)
        state = model.state_dict
        print(state().keys())
        #print(model.c_layer5)
        for name, module in model.named_children():
            print(name)
        #t_pooler_layer = state()['model.bert.t_pooler.dense.weight'] + state()["model.bert.t_pooler.dense.bias"]
        t_pooler_layer = state()["model.classifier.0.dense.weight"] + state()["model.classifier.0.dense.bias"]
        #maybe register forward hook to get the output???
        idxs = generate_step(t_pooler_layer, gen_idx=seed_len+ii, top_k=top_k, temperature=temperature, sample=sample)
        for jj in range(batch_size):
            prepared_batch.get_field("input_ids")[jj][seed_len+ii] = idxs

    return untokenize_batch(prepared_batch.get_field("input_ids"))


def generate(n_samples, seed_text="[CLS]", batch_size=10, max_len=25, 
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    start_time = time.time()
    batch_n = 0
    for original_batch in tqdm.tqdm(dataloader):
        #batch = parallel_sequential_generation(original_batch, seed_text, max_len=max_len, top_k=top_k,
        #                                       temperature=temperature, burnin=burnin, max_iter=max_iter, 
        #                                       cuda=cuda, verbose=True)
        
        batch = sequential_generation(original_batch, seed_text, batch_size=1, max_len=max_len, top_k=top_k, temperature=temperature, leed_out_len=leed_out_len, sample=sample)
        #batch = parallel_generation(seed_text, max_len=max_len, top_k=top_k, temperature=temperature, sample=sample, max_iter=max_iter)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        batch_n += 1
        sentences += batch
        print(batch)
        if batch_n > 2:
            break
    return sentences

In [12]:

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))
    
def read_sents(in_file, should_detokenize=False):
    sents = [sent.strip().split() for sent in open(in_file).readlines()]
    if should_detokenize:
        sents = [detokenize(sent) for sent in sents]
    return sents

def write_sents(out_file, sents, should_detokenize=False):
    with open(out_file, "w") as out_fh:
        for sent in sents:
            sent = detokenize(sent[1:-1]) if should_detokenize else sent
            out_fh.write("%s\n" % " ".join(sent))

In [13]:
n_samples = 1000
batch_size = 50
max_len = 40
top_k = 100
temperature = 0.7

leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500
registry.register("current_device", torch.device('cuda'))

# Choose the prefix context
seed_text = "[CLS]".split()

with torch.no_grad():
    for temp in [1.0]:
        model_version = "v1"
        bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                              sample=sample, top_k=top_k, temperature=temp, burnin=burnin, max_iter=max_iter,
                              cuda=False)
        out_file = "data/%s-len%d-burnin%d-topk%d-temp%.3f.txt" % (model_version, max_len, burnin, top_k, temp)
        write_sents(out_file, bert_sents, should_detokenize=True)

  0%|          | 0/8500 [00:00<?, ?it/s]


TypeError: device must be either 'str' or 'torch.device' type, <class 'NoneType'> found

In [ ]:
print(list(tokenizer.vocab.keys())[5000])